# Занятие 4. Деревья, представления и обходы

In [1]:
# Замена стандартного метода input() для чтения из строки
from utils.string_input import input, set_input, set_file

---

## A. Родословная: подсчет уровней

In [2]:
set_input("""
9
Alexei Peter_I
Anna Peter_I
Elizabeth Peter_I
Peter_II Alexei
Peter_III Anna
Paul_I Peter_III
Alexander_I Paul_I
Nicholaus_I Paul_I
""")

n = int(input())
parents = dict(input().split() for _ in range(n-1))

for name in sorted({*parents} | {*parents.values()}):
    parent = name
    for i in range(n):
        if parent not in parents:
            break
        parent = parents[parent]
    print(name, i)

Alexander_I 4
Alexei 1
Anna 1
Elizabeth 1
Nicholaus_I 4
Paul_I 3
Peter_I 0
Peter_II 2
Peter_III 2


---

## B. Родословная: число потомков

In [95]:
set_input('''
9
Alexei Peter_I
Anna Peter_I
Elizabeth Peter_I
Peter_II Alexei
Peter_III Anna
Paul_I Peter_III
Alexander_I Paul_I
Nicholaus_I Paul_I
''')

from collections import defaultdict
import sys


sys.setrecursionlimit(1000000)
n = int(input())
tree = defaultdict(list)
descendants = defaultdict(int)
children = set()

for _ in range(n-1):
    child, parent = input().split()
    children.add(child)
    tree[parent].append(child)

def count_descendants(node):
    for child in tree[node]:
        descendants[node] += count_descendants(child)
    return descendants[node] + 1

count_descendants(({*tree} - children).pop())
for name in sorted(tree):
    print(name, descendants[name])

Alexander_I 0
Alexei 1
Anna 4
Elizabeth 0
Nicholaus_I 0
Paul_I 2
Peter_I 8
Peter_II 0
Peter_III 3


---

## C. Родословная: LCA

In [13]:
set_file("""
9
Alexei Peter_I
Anna Peter_I
Elizabeth Peter_I
Peter_II Alexei
Peter_III Anna
Paul_I Peter_III
Alexander_I Paul_I
Nicholaus_I Paul_I
Alexander_I Nicholaus_I
Peter_II Paul_I
Alexander_I Anna
""")

with open('input.txt', 'r') as file:
    lines = [line.strip() for line in file]

n = int(lines[0])
tree, parents = {}, {}

for line in lines[1:n]:
    child, parent = line.split()
    tree[parent] = tree.get(parent, []) + [child]
    parents[child] = parent

def path_to_root(name):
    path = []
    while name in parents:
        path.append(name)
        name = parents[name]
    return path + [name]

def find_lca(name1, name2):
    path1 = path_to_root(name1)
    path2 = path_to_root(name2)
    lca = None
    for a, b in zip(path1[::-1], path2[::-1]):
        if a == b:
            lca = a
        else:
            break
    return lca

for line in lines[n:]:
    print(find_lca(*line.split()))

Paul_I
Peter_I
Anna


---

## D. Бинарное дерево (вставка, поиск, обход)

In [3]:
set_file("""
ADD 2
ADD 3
ADD 2
SEARCH 2
ADD 5
PRINTTREE
SEARCH 7
""")

class Node:
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None


class Tree:
    def __init__(self):
        self.root = None

    def add(self, value):
        if self.root is None:
            self.root = Node(value)
            print('DONE')
        else:
            self._add(self.root, value)

    def _add(self, current, value):
        if value < current.value:
            if current.left is None:
                current.left = Node(value)
                print('DONE')
            else:
                self._add(current.left, value)
        elif value > current.value:
            if current.right is None:
                current.right = Node(value)
                print('DONE')
            else:
                self._add(current.right, value)
        else:
            print('ALREADY')

    def search(self, value):
        self._search(self.root, value)

    def _search(self, current, value):
        if current is None:
            print('NO')
        elif current.value == value:
            print('YES')
        elif value < current.value:
            self._search(current.left, value)
        else:
            self._search(current.right, value)

    def display(self):
        self._display(self.root, 0)

    def _display(self, current, depth):
        if current is not None:
            self._display(current.left, depth+1)
            print('.'*depth + str(current.value))
            self._display(current.right, depth+1)


tree = Tree()
with open('input.txt', 'r') as file:
    for command in file:
        match command.strip().split():
            case 'ADD', value:
                tree.add(int(value))
            case 'SEARCH', value:
                tree.search(int(value))
            case 'PRINTTREE',:
                tree.display()

DONE
DONE
ALREADY
YES
DONE
2
.3
..5
NO


---

## E. Размер поддеревьев

In [47]:
set_input("""
7
1 2
1 3
1 4
5 1
6 5
7 5
""")

from collections import defaultdict
import sys

sys.setrecursionlimit(100000)
n = int(input())
rels, counts = defaultdict(list), {}

for _ in range(n-1):
    node1, node2 = map(int, input().split())
    rels[node1].append(node2)
    rels[node2].append(node1)

def count_vertices(node, skip=-1):
    count = 0
    for rel in rels.get(node, []):
        if rel != skip:
            count += 1 + count_vertices(rel, node)
    counts[node] = count
    return count

count_vertices(1)
print(*[counts.get(node + 1, 0) + 1 
        for node in range(n)])

7 1 1 1 3 1 1


---

## F. Бюрократия

In [2]:
set_input("""
5
1 2 2 4
""")

from collections import defaultdict
import sys

sys.setrecursionlimit(1000000)
n = int(input())
parents = map(int, input().split())
tree = defaultdict(list)
tasks = [1] * (n + 1)
money = [1] * (n + 1)

for child, parent in enumerate(parents, 2):
    tree[parent].append(child)

def count_tasks(node=1):
    for child in tree[node]:
        tasks[node] += count_tasks(child)
    return tasks[node] 

def count_money(node=1):
    for child in tree[node]:
        money[node] += count_money(child) + tasks[child]
    return money[node]

count_tasks()
count_money()

print(*money[1:])

13 8 1 3 1


---

## G. Дятлы

In [54]:
set_input("""
4 2 13
1 2
2 3
""")

from collections import defaultdict
import sys

# Чтение параметров графа отношений между дятлами
N, M, K = map(int, input().split())
tree = defaultdict(list)
for _ in range(M):
    node1, node2 = map(int, input().split())
    tree[node1].append(node2)
    tree[node2].append(node1)

# Разобрать компоненту связности и найти ее листья
def parse_graph(node):
    leafs = []
    stack = [(node, 0)]
    while stack:
        current, parent = stack.pop()
        if current in visited:
            print(0)
            sys.exit()
        visited.add(current)
        counter = [0, 0]
        for child in tree[current]:
            if child != parent:
                is_multi = len(tree[child]) > 1
                counter[is_multi] += 1
                if counter[1] > 2 - (parent > 0):
                    print(0)
                    sys.exit()
                if is_multi:
                    stack.append((child, current))
        leafs.append(counter[0])
    return leafs

# Выделение компонентов связности у графа
graphs = []
visited = set()
for node in tree:
    if node not in visited:
        if len(tree[node]) != 1:
            graphs.append(parse_graph(node))
        elif len(tree[tree[node][0]]) == 1:
            visited.add(node)
            visited.add(tree[node][0])
            graphs.append([1])

# Количество дятлов, имеющих связи, и количество дятлов-одиночек
n_linked = len(tree)
n_loners = N - n_linked

# Подсчет факториала (сразу по модулю) заранее
fact_l = [1] + [0] * (N+1)
fact_r = [0] * (N+1) + [N+1, 1]
for i in range(N+1):
    fact_l[i+1] = (fact_l[i] * (i+1)) % K
    fact_r[N-i] = (fact_r[N-i+1] * (N-i)) % K

answer = 1
# Количество перестановок компонент связности
answer *= fact_l[len(graphs)]
for graph in graphs:
    for n in graph:
        # Количество перестановок листьев в компонентах связности
        answer = (answer * fact_l[n]) % K
    # Количество способов отразить компоненту связности
    answer = (answer * (4, 2)[len(graph) == 1]) % K
# Количество перестановок дятлов-одиночек
answer = (answer * fact_r[n_linked + 2]) % K

print(answer)

7


---

## H. Вершинно-реберное покрытие дерева *

In [5]:
set_input("""
6
1 2
2 3
1 4
4 5
4 6
22 48 2 2 8 1
""")
    
from collections import deque

# Чтение дерева
N = int(input())
tree = [set() for _ in range(N+1)]
for _ in range(N - 1):
    node1, node2 = map(int, input().split())
    tree[node1].add(node2)
    tree[node2].add(node1)

# Чтение стоимостей пометок вершин
costs = [0]
for cost in map(int, input().split()):
    costs.append(cost)

# Если узел всего один, его обязательно нужно пометить
if N == 1:
    print(costs[1], 1)
    print(1)
else:
    # Динамика для стоимости, если i-ую вершину не пометить
    dp_skip = [0] * (N+1)
    # Динамика для стоимости, если i-ую вершину пометить
    dp_mark = [0] * (N+1)

    # Поиск всех листьев
    leafs = deque([])
    for i in range(1, N+1):
        if len(tree[i]) == 1:
            leafs.append(i)
            dp_mark[i] = costs[i]

    # Узлы-истоки от пути с листов для каждой вершины
    parents = [set() for _ in range(N+1)]
    
    # Начиная с каждого листа, пройтись по дереву и заполнить dp
    while leafs:
        leaf = leafs.popleft()
        # Идем вперед по дереву до вершины-перекрестка
        while len(tree[leaf]) - len(parents[leaf]) <= 1:
            if parents[leaf]:
                # Динамика
                to_mark = to_skip = 0
                for parent in parents[leaf]:
                    to_mark += min(dp_mark[parent], dp_skip[parent])
                    to_skip += dp_mark[parent]
                dp_mark[leaf] = to_mark + costs[leaf]
                dp_skip[leaf] = to_skip
            # Если пришли в лист, то алгоритм закончен
            if leaf == leafs[0]:
                leaf = leafs.pop()
                break
            # Продвигаемся по дереву дальше
            for neig in tree[leaf]:
                if neig not in parents[leaf]:
                    parents[neig].add(leaf)
                    leaf = neig
                    break

    # Восстановление всех помеченных вершин
    marks = set()
    stack = [(leaf, -1)]
    while stack:
        node, prev = stack.pop()
        if prev == -1 or prev in marks:
            if dp_mark[node] < dp_skip[node]:
                marks.add(node)
        else:
            marks.add(node)
        for neig in parents[node]:
            stack.append((neig, node))

    # Вывод ответа
    if dp_skip[leaf] < dp_mark[leaf]:
        print(dp_skip[leaf], end=' ')
    else:
        print(dp_mark[leaf], end=' ')
    print(len(marks))
    print(*marks)

26 3
1 3 4


---

## I. Пара путей *

In [3]:
set_input("""
12
1 2
2 3
3 4
3 6
6 5
6 7
7 8
8 9
9 10
10 11
5 12
""")

from collections import deque
import sys

# Чтение дерева
sys.setrecursionlimit(1000000)
N = int(input())
tree = [[] for _ in range(N+1)]
for _ in range(N - 1):
    node1, node2 = map(int, input().split())
    tree[node1].append(node2)
    tree[node2].append(node1)

# Обход в ширину для поиска диаметра
def bfs(start):
    dist = [-1] * (N + 1)
    parents = [-1] * (N + 1)
    dist[start] = 0
    queue = deque([start])
    farthest_node = start
    max_dist = 0
    while queue:
        current = queue.popleft()
        for neighbor in tree[current]:
            if dist[neighbor] == -1:
                dist[neighbor] = dist[current] + 1
                parents[neighbor] = current
                queue.append(neighbor)
                if dist[neighbor] > max_dist:
                    max_dist = dist[neighbor]
                    farthest_node = neighbor
    return farthest_node, max_dist, parents

# Поиск диаметра
node1, *_ = bfs(1)
node2, diameter, parents = bfs(node1)

# Составление пути по диаметру
current = node2
path = []
while current != -1:
    path.append(current)
    current = parents[current]

# Вычислить максимальное расстояние до листа
def find_max_dist(node, parent):
    if max_ds[node] == -1:
        if len(tree[node]) == 1:
            max_ds[node] = 0
        else:
            ds = []
            for neighbor in tree[node]:
                if neighbor != parent:
                    ds.append(find_max_dist(neighbor, node))
            max_ds[node] = max(ds) + 1
    return max_ds[node]

# Посчитать максимальные расстояния до листьев
max_ds = [-1] * (N+1)
for i in range(1, len(path)-1):
    for neighbor in tree[path[i]]:
        if neighbor not in (path[i-1], path[i+1]):
            find_max_dist(neighbor, path[i])

# Среди дистанций до узла найти наибольший путь
def max_path(dists):
    if len(dists) == 0:
        return 0
    if len(dists) == 1:
        return dists[-1]
    return sum(sorted(dists, reverse=True)[:2]) + 2

max_prod = 0
max_d_before = 0
for i in range(2, len(path)-1):
    # Когда разрез делается между двумя точками на диаметре
    l, r = i-1, len(path)-i-1
    d1 = max_path([l-1] + [max_ds[neig] for neig in tree[path[i-1]] if neig != path[i]])
    d2 = max_path([r-1] + [max_ds[neig] for neig in tree[path[i]] if neig != path[i-1]])
    max_d_before = max(d1, max_d_before)
    max_prod = max(d1 * d2, max_prod)
    max_prod = max(d2*max_d_before, max_prod)
    # Когда разрез делается между точкой на диаметре и другой точкой
    for neighbor in tree[path[i]]:
        if neighbor not in (path[i-1], path[i+1]):
            d1 = max_path([max_ds[neig] for neig in tree[neighbor] if neig != path[i]])
            d2 = diameter
            max_d_before = max(d1, max_d_before)
            max_prod = max(d1*d2, max_prod)

print(max_prod)

21


---

## J. Количество топсортов дерева *

In [ ]:
set_input("""

""")